In [2]:
import pandas as pd
import numpy as np
import math
import datetime

pd.options.mode.chained_assignment = None

Link database:
https://drive.google.com/drive/folders/1C6VFBCvsIt_MclJ-aKVI-rJBof0k7iDf?usp=drive_link

In [ ]:
database_1 = pd.read_csv('/Users/irfanhilman/Downloads/Database/Database Ringkas/Database Part 1.csv')
database_2 = pd.read_csv('/Users/irfanhilman/Downloads/Database/Database Ringkas/Database Part 2.csv')
database_3 = pd.read_csv('/Users/irfanhilman/Downloads/Database/Database Ringkas/Database Part 3.csv')
database_4 = pd.read_csv('/Users/irfanhilman/Downloads/Database/Database Ringkas/Database Part 4.csv')
database = pd.concat([database_1,database_2,database_3,database_4])
database = database.drop(columns='Unnamed: 0')

#Substitusi nilai Open, High, Low Price yg 0 dengan Close Price
database['Open Price'] = np.where(database['Open Price'].eq(0),database['Close Price'],database['Open Price'])
database['High Price'] = np.where(database['High Price'].eq(0),database['Close Price'],database['High Price'])
database['Low Price'] = np.where(database['Low Price'].eq(0),database['Close Price'],database['Low Price'])

In [ ]:
port_performance = pd.read_csv('/Users/irfanhilman/Downloads/Database/backtesting_pegadaian.csv')


In [ ]:
list_port_performance = []
for i in list(port_performance['Date'].unique()):
    port_performance_i = port_performance[port_performance['Date']==i]
    if port_performance_i['Action'].unique()[0] == 'SELL':
        port_performance_i['Weight'] = port_performance_i['Sold']/sum(port_performance_i['Sold'])
    else: pass
    list_port_performance.append(port_performance_i)
port_performance_ = pd.concat(list_port_performance)

In [ ]:
rolling_dic = {'1 Month':20,'3 Month':60,'6 Month':120,'1 Year':240}
def rolling_average(df,param,period):
    if f'Average {param} {period}' in df.columns:
        df = df.drop(columns=f'Average {param} {period}')
    else: pass
    init_list = ['Kode','Date']
    init_list.append(param)
    df['Kode + Date'] = df['Kode'] + df['Date'].astype('str')
    df_ = df[init_list]
    list_df = []
    for i in set(list(df_['Kode'])):
        df_i = df_[df_['Kode']==i]
        df_i[f'Average {param} {period}'] = df_i[f'{param}'].rolling(rolling_dic[period],min_periods=1).mean() # Min 1 data point
        list_df.append(df_i)
    df__ = pd.concat(list_df).sort_values(by=['Kode','Date'])
    df__['Kode + Date'] = df__['Kode'].astype('str') + df__['Date'].astype('str')
    df__ = df__.drop_duplicates(subset='Kode + Date')
    df = df.merge(df__[[f'Average {param} {period}','Kode + Date']],on='Kode + Date',how='left')
    return df

def rolling_stdev(df,param,period):
    if f'Stdev {param} {period}' in df.columns:
        df = df.drop(columns=f'Stdev {param} {period}')
    else: pass
    init_list = ['Kode','Date']
    init_list.append(param)
    df['Kode + Date'] = df['Kode'] + df['Date'].astype('str')
    df_ = df[init_list]
    list_df = []
    for i in set(list(df_['Kode'])):
        df_i = df_[df_['Kode']==i]
        df_i[f'Stdev {param} {period}'] = df_i[f'{param}'].rolling(rolling_dic[period],min_periods=1).std() # Min 1 data point
        list_df.append(df_i)
    df__ = pd.concat(list_df).sort_values(by=['Kode','Date'])
    df__['Kode + Date'] = df__['Kode'].astype('str') + df__['Date'].astype('str')
    df__ = df__.drop_duplicates(subset='Kode + Date')
    df = df.merge(df__[[f'Stdev {param} {period}','Kode + Date']],on='Kode + Date',how='left')
    return df

In [ ]:

database = rolling_average(df=database,param='Momentum 1 Day',period='1 Year')
database = rolling_stdev(df=database,param='Momentum 1 Day',period='1 Year')

In [ ]:
port_performance_ = port_performance_[['Day Number','Date','Kode','Weight','Total Capital']]
port_performance_['Kode + Date'] = port_performance_['Kode'].astype('str') + ' ' + port_performance_['Date'].astype('str')
database['Kode + Date'] = database['Kode'].astype('str') + ' ' + database['Date'].astype('str')
port_performance_ = port_performance_.merge(database[['Kode + Date','Average Momentum 1 Day 1 Year','Stdev Momentum 1 Day 1 Year']],on='Kode + Date',how='left')
port_performance_ = port_performance_.drop(columns='Kode + Date')
database = database.drop(columns='Kode + Date')
port_performance_ = port_performance_.rename(columns={'Average Momentum 1 Day 1 Year':'Expected Return 1 Day in 1 Year','Stdev Momentum 1 Day 1 Year':'Stdev Return 1 Day in 1 Year'})

In [ ]:
day_numbers = 21*3 #number of days for end portfolio value
monte_carlo_iter = 10000 #number of iteration in MC simulation
confidence_interval = [0.9,0.95]
database_sh = database[['Date','Kode','Momentum 1 Day']]
port_performance__i_list = []
for i in list(port_performance_['Day Number'].unique()):
    print(i)
    port_performance__i = port_performance_[port_performance_['Day Number']==i]
    port_performance__date = port_performance__i['Date'].unique()[0]
    port_performance__kode = port_performance__i[['Kode']]
    database_sh_i = port_performance__kode.merge(database_sh,on='Kode',how='left')
    database_sh_i_j_list = []
    for j in list(database_sh_i['Kode'].unique()):
        database_sh_i_j = database_sh_i[database_sh_i['Kode']==j]
        database_sh_i_j = database_sh_i_j[['Date','Momentum 1 Day']]
        database_sh_i_j = database_sh_i_j.rename(columns={'Momentum 1 Day':j})
        database_sh_i_j = database_sh_i_j.drop_duplicates(subset='Date')
        database_sh_i_j = database_sh_i_j.set_index('Date')
        database_sh_i_j_list.append(database_sh_i_j)
    database_kode = pd.concat(database_sh_i_j_list,axis=1)
    database_kode = database_kode.sort_index()
    database_kode = database_kode.iloc[-252:]
    database_kode = database_kode.fillna(0)
    covariance_matrix = database_kode.cov().values
    weights =  port_performance__i['Weight']
    portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    portfolio_stdev = portfolio_variance**(0.5)
    portfolio_exp_return = sum(port_performance__i['Weight']*port_performance__i['Expected Return 1 Day in 1 Year'])
    portfolio_total_capital = port_performance__i['Total Capital'].iloc[0]
    portfolio_total_capital_list = []
    for k in range(monte_carlo_iter):
        portfolio_total_capital_ = portfolio_total_capital
        for l in range(day_numbers):
            portfolio_total_capital_ = portfolio_total_capital_*(1+np.random.normal(portfolio_exp_return, portfolio_stdev))
        portfolio_total_capital_list.append(portfolio_total_capital_)
    var_1 = np.percentile(portfolio_total_capital_list, 100*(1-confidence_interval[0]))
    var_2 = np.percentile(portfolio_total_capital_list, 100*(1-confidence_interval[1]))
    port_performance__i[f'VAR {day_numbers} Days at {confidence_interval[0]} CI'] = var_1
    port_performance__i[f'VAR {day_numbers} Days at {confidence_interval[1]} CI'] = var_2
    port_performance__i[f'VAR {confidence_interval[0]} - Total Capital'] = var_1 - port_performance__i['Total Capital']
    port_performance__i[f'VAR {confidence_interval[1]} - Total Capital'] = var_2 - port_performance__i['Total Capital']
    port_performance__i[f'VAR {confidence_interval[0]} Deviation'] = port_performance__i[f'VAR {confidence_interval[0]} - Total Capital'] /port_performance__i['Total Capital']
    port_performance__i[f'VAR {confidence_interval[1]} Deviation'] = port_performance__i[f'VAR {confidence_interval[1]} - Total Capital'] /port_performance__i['Total Capital']
    port_performance__i_list.append(port_performance__i)
port_var = pd.concat(port_performance__i_list)
port_var = port_var.sort_values(by='Date')
port_var = port_var.reset_index(drop=True)
    